# Using the simulator to track ice volumes

In this tutorial, we use the `Tracker` functionality of the Layer Tracer Simulator to track the progression of a block of ice.
See `layer_tracer_tutorial` for details on the simulator.

## Loading Posterior

In this tutorial, we set the accumulation and melting to be that of the posterior maximum-a-posterior (MAP) estimate,
which is the most likely model from our posterior. For this, we begin by first loading the posterior.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sbi.utils import get_density_thresholder, RestrictedPrior, BoxUniform,process_prior
from torch.distributions import MultivariateNormal
import os
from omegaconf import DictConfig,OmegaConf
from sbi_ice.loaders import ShortProfileLoader
import sbi.analysis as analysis
from sbi_ice.utils import posterior_utils,modelling_utils,plotting_utils,misc
from sbi_ice.simulators import Layer_Tracing_Sim as lts
import sbi_ice.utils.noise_model as noise_model


data_dir,output_dir = misc.get_data_output_dirs()
work_folder = misc.get_project_root()
color_opts = plotting_utils.setup_plots()


In [ ]:
shelf = "Ekstrom"
exp = "exp2"
name = "all_final"
seed = "layer_0_seed_101" #top layer
fol = Path(output_dir , shelf, exp, "sbi_sims/post_predictives" , name,seed)
cfg_path = Path(fol,"config.yaml")
cfg = OmegaConf.load(cfg_path)
config_fol = cfg.posterior_config_fol
config_fol = Path(output_dir,shelf,exp,"sbi_sims/posteriors",name)
print("Path to config file: " , Path(config_fol,str(cfg.name),"config.yaml"))


n_post_samples = cfg.n_post_samples
n_predictive_sims = cfg.n_predictive_sims
print("number of predictive sims loaded: " , n_predictive_sims)
overwrite = cfg.overwrite_saved_sims
posterior_config = OmegaConf.load(Path(config_fol,str(cfg.name),"config.yaml"))
print("Path to posterior config file: " ,posterior_config)


paths = posterior_config.paths
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_fol = os.getcwd()
print("Path to output folder where sims are stored: ", output_fol)

shelf_setup_path = paths.shelf_setup_path
print("Setup file for shelf: ", shelf_setup_path)


shelf_folder_path = paths.shelf_folder_path
exp_path = paths.exp_path

print("Read Config files")
loader = ShortProfileLoader.ShortProfileLoader(Path(work_folder,shelf_setup_path),Path(work_folder,shelf_folder_path),exp_path,sims_path="layer_sims",gt_version = posterior_config.gt_version)
loader._jobs = [i for i in range(1,200)]
loader._num_sims = [1000 for i in range(1,200)]
loader.total_sims = 1000*200
print("Set up data loader")

In [ ]:
with open(Path(config_fol,str(cfg.name),"inference.p"), "rb") as f:
    if device.type == "cpu":
        out = posterior_utils.CPU_Unpickler(f).load()
    else:
        out = pickle.load(f)


inference = out["inference"]
prior = out["prior"]
layer_mask = out["layer_mask"]
smb_mask = out["smb_mask"]
true_layer = torch.tensor(loader.real_layers[cfg.layer_idx][layer_mask]).float()
posterior = inference.build_posterior(inference._neural_net.to("cpu"))
posterior.set_default_x(true_layer)
samples = posterior.sample((n_post_samples,))

posterior_smb_map = posterior.map()

print(posterior_smb_map)

# Defining Simulator

Now that we have the accumulation we want to run the model with, we can set up and run the simulator with this parameter.



In [ ]:
name = "Ekstrom"
data_file = Path(data_dir,name,"setup_files","flowtube_full.csv")


df = pd.read_csv(data_file)
xmb = df["x_coord"].to_numpy() #x - coordinates of domain
tmb = df["tmb"].to_numpy() #total mass balance
Lx = xmb.max() - xmb.min()

nx_iso             = 500 # Number of points in the x-direction
ny_iso             = 1 # Number of points in the y-direction
dt                 = 0.5 # [yr] timestep for advection scheme


geom = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso) #This will be the main object we interact with

## Interpolating onto simulation grid
# The resolution we infer the smb for is coarser than what we want to simulate with.
# As a rough approximation, let us define the posterior mean smb on the finer simulation grid by interpolation.
smb_regrid = modelling_utils.regrid(loader.x[smb_mask],posterior_smb_map.detach().numpy(),xmb)



smb_regrid,bmb_regrid = lts.init_geom_from_fname(geom,data_file,smb=smb_regrid)

fig,ax = plt.subplots(1,1,figsize = (4,2))
ax.plot(xmb/1000,smb_regrid,color = plotting_utils.color_opts["colors"]["prior"],alpha = 0.3)
ax.set_xlabel("Distance from GL [km]")
ax.set_ylabel("posterior SMB MAP [m/yr]")


In [ ]:
n_surface_phase1 = 25
n_base_phase1 = 100
time_phase1 = 2000
n_surface_phase2 = 2
n_base_phase2 = 50
time_phase2 = 1000

sched1 = lts.Scheduele(time_phase1,dt,n_surface_phase1,n_base_phase1)
sched2 = lts.Scheduele(time_phase2,dt,n_surface_phase2,n_base_phase2)
print(sched1.total_iterations)
print(sched2.total_iterations)

In [ ]:
#Now simulate

geom.initialize_layers(sched2,100,tracker_coords=np.array([[0.0,0.0],[0.0,100.0],[0.0,150.0]])) #trackers are seeded in (x,depth) coordinates
# lts.sim(geom,smb_regrid,bmb_regrid,sched1)
# #Can save intermediate results here if we want
# geom.initialize_layers(sched2,0)
# lts.sim(geom,smb_regrid,bmb_regrid,sched2)

# Initial Simulator Setup

In [ ]:
fig,axs = plotting_utils.plot_isochrones_1d(geom.x,geom.bs.flatten(),geom.ss.flatten(),geom.dsum_iso[:,0,::10],geom.age_iso[::10],bmb_regrid,smb_regrid,real_layers=None,trackers=None)
trackers = geom.extract_active_trackers()
print(trackers)
axs[1].scatter(trackers[:,0]/1000,trackers[:,2],color = "brown")

# Simulate

In [ ]:
geom.initialize_layers(sched1,10)
lts.sim(geom,smb_regrid,bmb_regrid,sched1)
#Can save intermediate results here if we want
geom.initialize_layers(sched2,0)
lts.sim(geom,smb_regrid,bmb_regrid,sched2)

In [ ]:
fig,axs = plotting_utils.plot_isochrones_1d(geom.x,geom.bs.flatten(),geom.ss.flatten(),geom.dsum_iso[:,0,::10],geom.age_iso[::10],bmb_regrid,smb_regrid,real_layers=None,trackers=None)
trackers = geom.extract_active_trackers()
print(trackers)
axs[1].scatter(trackers[::25,0]/1000,trackers[::25,2],color = "brown")
# axs[1].set_xlim(-5,10)
# axs[1].set_ylim(-50,150)

# Extract data

In [ ]:
data = np.array([trackers[-1::-3,0],trackers[-1::-3,2],trackers[-2::-3,0],trackers[-2::-3,2],trackers[-3::-3,0],trackers[-3::-3,2]]).T
print(data.shape) 
tracker_df = pd.DataFrame(data=data,
                  columns= ["P1_x","P1_z","P2_x","P2_z","P3_x","P3_z"])

print(tracker_df)
tracker_df.to_csv(Path(output_dir,"Ekstrom","scratch","trackers.csv"),index=False)

In [ ]:
out_df = df.copy()
out_df["bmb"] = bmb_regrid
out_df["smb"] = smb_regrid
out_df.to_csv(Path(output_dir,"Ekstrom","scratch","flowtube_full.csv"),index=False)

In [ ]:
out_df